# Unit Testing `GiRaFFE_NRPy`: $A_k$ to $B^i$

### Author: Patrick Nelson

This notebook validates our A-to-B solver for use in `GiRaFFE_NRPy`. Because the original `GiRaFFE` used staggered grids and we do not, we can not trivially do a direct comparison to the old code. Instead, we will compare the numerical results with the expected analytic results. 

**Module Status:** <font color=red><b> In-Progress </b></font>

**Validation Notes:** This module will validate the routines in [Tutorial-GiRaFFE_HO_C_code_library-A2B](../Tutorial-GiRaFFE_HO_C_code_library-A2B.ipynb).

It is, in general, good coding practice to unit test functions individually to verify that they produce the expected and intended output. Here, we expect our functions to produce the correct cross product in an arbitrary spacetime. To that end, we will choose functions that are easy to differentiate, but lack the symmetries that would trivialize the finite-difference algorithm. Higher-order polynomials are one such type of function. 

We will start with the simplest case - testing the second-order solver. In second-order finite-differencing, we use a three-point stencil that can exactly differentiate polynomials up to quadratic. So, we will use cubic functions three variables. For instance,

\begin{align}
A_x &= ax^3 + by^3 + cz^3 + dy^2 + ez^2 + f \\
A_y &= gx^3 + hy^3 + lz^3 + mx^2 + nz^2 + p \\
A_z &= px^3 + qy^3 + rz^3 + sx^2 + ty^2 + u. \\
\end{align}

It will be much simpler to let NRPy+ handle most of this work. So, we will import the core functionality of NRPy+, build the expressions, and then output them using `outputC()`.

In [1]:
import shutil, os, sys           # Standard Python modules for multiplatform OS-level functions
# First, we'll add the parent directory to the list of directories Python will check for modules.
nrpy_dir_path = os.path.join("..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
nrpy_dir_path = os.path.join("..","..")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)

from outputC import *            # NRPy+: Core C code output module
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import grid as gri               # NRPy+: Functions having to do with numerical grids
import loop as lp                # NRPy+: Generate C code loops
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

out_dir = "Validation/"
cmd.mkdir(out_dir)

thismodule = "Unit_Test_GiRaFFE_NRPy_Ccode_library_A2B"
a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u = par.Cparameters("REAL",thismodule,["a","b","c","d","e","f","g","h","l","m","n","o","p","q","r","s","t","u"],10.0)
gammadet = gri.register_gridfunctions("AUXEVOL","gammadet")

DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

par.set_parval_from_str("reference_metric::CoordSystem","Cartesian")
rfm.reference_metric()
x = rfm.xxCart[0]
y = rfm.xxCart[1]
z = rfm.xxCart[2]

AD = ixp.register_gridfunctions_for_single_rank1("EVOL","AD")
AD[0] = a*x**3 + b*y**3 + c*z**3 + d*y**2 + e*z**2 + f
AD[1] = g*x**3 + h*y**3 + l*z**3 + m*x**2 + n*z**2 + o
AD[2] = p*x**3 + q*y**3 + r*z**3 + s*x**2 + t*y**2 + u


We also need to create the files that interact with NRPy's C parameter interface. (This is for the newer version of the A2B routine, as we replace the old with it in this unit test.)

In [2]:
# Step 3.d.i: Generate declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(out_dir))

# Step 3.d.ii: Set free_parameters.h
with open(os.path.join(out_dir,"free_parameters.h"),"w") as file:
    file.write("""
// Override parameter defaults with values based on command line arguments and NGHOSTS.
// We'll use this grid. It has one point and one ghost zone.
params.Nxx0 = atoi(argv[1]);
params.Nxx1 = atoi(argv[2]);
params.Nxx2 = atoi(argv[3]);
params.Nxx_plus_2NGHOSTS0 = params.Nxx0 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS1 = params.Nxx1 + 2*NGHOSTS;
params.Nxx_plus_2NGHOSTS2 = params.Nxx2 + 2*NGHOSTS;
// Step 0d: Set up space and time coordinates
// Step 0d.i: Declare \Delta x^i=dxx{0,1,2} and invdxx{0,1,2}, as well as xxmin[3] and xxmax[3]:
const REAL xxmin[3] = {-0.01,-0.01,-0.01};
const REAL xxmax[3] = { 0.01, 0.01, 0.01};

params.dxx0 = (xxmax[0] - xxmin[0]) / ((REAL)params.Nxx0);
params.dxx1 = (xxmax[1] - xxmin[1]) / ((REAL)params.Nxx1);
params.dxx2 = (xxmax[2] - xxmin[2]) / ((REAL)params.Nxx2);
params.invdx0 = 1.0 / params.dxx0;
params.invdx1 = 1.0 / params.dxx1;
params.invdx2 = 1.0 / params.dxx2;
\n""")

# Generates declare_Cparameters_struct.h, set_Cparameters_default.h, and set_Cparameters[-SIMD].h
par.generate_Cparameters_Ccodes(os.path.join(out_dir))

Next, we'll let NRPy+ compute derivatives analytically according to $$B^i = \frac{[ijk]}{\sqrt{\gamma}} \partial_j A_k.$$ Then we can carry out two separate tests to verify the numerical derivatives. First, we will verify that when we let the cubic terms be zero, the two calculations of $B^i$ agree to roundoff error. Second, we will verify that when we set the cubic terms, our error is dominated by trunction error that converges to zero at the expected rate. 

In [3]:
import WeylScal4NRPy.WeylScalars_Cartesian as weyl
LeviCivitaDDD = weyl.define_LeviCivitaSymbol_rank3()
LeviCivitaUUU = ixp.zerorank3()
for i in range(DIM):
    for j in range(DIM):
        for k in range(DIM):
            LeviCivitaUUU[i][j][k] = LeviCivitaDDD[i][j][k] / sp.sqrt(gammadet)
            
B_analyticU = ixp.register_gridfunctions_for_single_rank1("AUXEVOL","B_analyticU")
for i in range(DIM):
    B_analyticU[i] = 0
    for j in range(DIM):
        for k in range(DIM):
            B_analyticU[i] += LeviCivitaUUU[i][j][k] * sp.diff(AD[k],rfm.xxCart[j])


We also should write a function that will use the analytic formulae for $B^i$. Then, we'll need to call the function from the module `GiRaFFE_HO_A2B` to generate the different header files. Also, we will declare the parameters for the vector potential functions.

In [4]:
B_analyticU_to_print = [\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU0"),rhs=B_analyticU[0]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU1"),rhs=B_analyticU[1]),\
                           lhrh(lhs=gri.gfaccess("out_gfs","B_analyticU2"),rhs=B_analyticU[2]),\
                          ]

desc = "Calculate the exact magnetic field"
name = "calculate_exact_BU"
outCfunction(
    outfile  = os.path.join(out_dir,"calculate_exact_BU.h"), desc=desc, name=name,
    params   ="const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs",
    preloop  ="""
        REAL xx0 = xx[0][i0];
        REAL xx1 = xx[1][i1];
        REAL xx2 = xx[2][i2];
""",
    body     = fin.FD_outputC("returnstring",B_analyticU_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts="AllPoints")

gri.glb_gridfcs_list = []
# cmd.mkdir(os.path.join(out_dir))
import GiRaFFE_NRPy.GiRaFFE_NRPy_A2B as A2B
# We'll generate these into the A2B subdirectory since that's where the functions
# we're testing expect them to be.
A2B.GiRaFFE_NRPy_A2B(os.path.join(out_dir))


Output C function calculate_exact_BU() to file Validation/calculate_exact_BU.h


We'll now write a function to set the vector potential $A_k$. This simply uses NRPy+ to generte most of the code from the expressions we wrote at the beginning. 

In [5]:
AD_to_print = [\
               lhrh(lhs=gri.gfaccess("out_gfs","AD0"),rhs=AD[0]),\
               lhrh(lhs=gri.gfaccess("out_gfs","AD1"),rhs=AD[1]),\
               lhrh(lhs=gri.gfaccess("out_gfs","AD2"),rhs=AD[2]),\
              ]

desc = "Calculate the vector potential"
name = "calculate_AD"
outCfunction(
    outfile  = os.path.join(out_dir,"calculate_AD.h"), desc=desc, name=name,
    params   ="const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs",
    preloop  ="""
        REAL xx0 = xx[0][i0];
        REAL xx1 = xx[1][i1];
        REAL xx2 = xx[2][i2];
""",
    body     = fin.FD_outputC("returnstring",AD_to_print,params="outCverbose=False").replace("IDX4","IDX4S"),
    loopopts="AllPoints")


Output C function calculate_AD() to file Validation/calculate_AD.h


Now that we have our vector potential and analytic magnetic field to compare against, we will start writing our unit test. We'll also import common C functionality, define `REAL`, the number of ghost zones, and the faces, and set the standard macros for NRPy+ style memory access.

In [6]:
%%writefile $out_dir/A2B_unit_test.C
// These are common packages that we are likely to need.
#include "stdio.h"
#include "stdlib.h"
#include "math.h"
#include "string.h" // Needed for strncmp, etc.
#include "stdint.h" // Needed for Windows GCC 6.x compatibility
#include <time.h>   // Needed to set a random seed.

#define REAL double
#include "declare_Cparameters_struct.h"

const int MAXFACE = -1;
const int NUL     = +0;
const int MINFACE = +1;
const int NGHOSTS = 1;

// Standard NRPy+ memory access:
#define IDX4(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * ( (k) + Nxx_plus_2NGHOSTS[2] * (g) ) ) )
#define IDX3(i,j,k) ( (i) + Nxx_plus_2NGHOSTS[0] * ( (j) + Nxx_plus_2NGHOSTS[1] * (k) ) )
// Assuming idx = IDX3(i,j,k). Much faster if idx can be reused over and over:
#define IDX4pt(g,idx)   ( (idx) + (Nxx_plus_2NGHOSTS[0]*Nxx_plus_2NGHOSTS[1]*Nxx_plus_2NGHOSTS[2]) * (g) )
#define IDX4S(g,i,j,k) \
( (i) + Nxx_plus_2NGHOSTS0 * ( (j) + Nxx_plus_2NGHOSTS1 * ( (k) + Nxx_plus_2NGHOSTS2 * (g) ) ) )


Overwriting Validation//A2B_unit_test.C


We'll now define the gridfunction names.

In [7]:
%%writefile -a $out_dir/A2B_unit_test.C
// Let's also #define the NRPy+ gridfunctions
#define AD0GF 0
#define AD1GF 1
#define AD2GF 2
#define NUM_EVOL_GFS 3

#define GAMMADETGF 0
#define B_ANALYTICU0GF 1
#define B_ANALYTICU1GF 2
#define B_ANALYTICU2GF 3
#define BU0GF 4
#define BU1GF 5
#define BU2GF 6
#define NUM_AUXEVOL_GFS 7



Appending to Validation//A2B_unit_test.C


Now, we'll handle the different A2B codes. There are several things to do here. First, we'll add `#include`s to the C code so that we have access to the functions we want to test. We must also create a directory and copy the files to that directory. We will choose to do this in the subfolder `A2B` relative to this tutorial.


In [8]:
%%writefile -a $out_dir/A2B_unit_test.C
#include "driver_AtoB.h" // This file contains both functions we need.

REAL a,b,c,d,e,f,g,h,l,m,n,o,p,q,r,s,t,u;

#include "calculate_exact_BU.h"
#include "calculate_AD.h"

Appending to Validation//A2B_unit_test.C


Now, we'll write the main method. First, we'll set up the grid. In this test, we cannot use only one point. As we are testing a three-point stencil, we can get away with a minimal $3 \times 3 \times 3$ grid. Then, we'll write the A fields. After that, we'll calculate the magnetic field two ways.

In [9]:
%%writefile -a $out_dir/A2B_unit_test.C
main(int argc, const char *argv[]) {
#include "set_Cparameters_default.h"

    // Let the first argument be the test we're doing. 1 = coarser grid, 0 = finer grid.
    int do_quadratic_test = atoi(argv[4]);
    
    // Step 0c: Set free parameters, overwriting Cparameters defaults 
    //          by hand or with command-line input, as desired.
#include "free_parameters.h"
#include "set_Cparameters-nopointer.h"

    // We'll define our grid slightly different from how we normally would. We let our outermost
    // ghostzones coincide with xxmin and xxmax instead of the interior of the grid. This means
    // that the ghostzone points will have identical positions so we can do convergence tests of them.    // Step 0d.ii: Set up uniform coordinate grids
    REAL *xx[3];
    for(int i=0;i<3;i++) {
        xx[i] = (REAL *)malloc(sizeof(REAL)*Nxx_plus_2NGHOSTS[i]);
        for(int j=0;j<Nxx_plus_2NGHOSTS[i];j++) {
            xx[i][j] = xxmin[i] + ((REAL)(j))*dxx[i]; // Face-centered grid.
        }
    }
    //for(int i=0;i<Nxx_plus_2NGHOSTS[0];i++) printf("xx[0][%d] = %.15e\\n",i,xx[0][i]);
    
    // This is the array to which we'll write the NRPy+ variables.
    REAL *auxevol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_AUXEVOL_GFS * Nxx_plus_2NGHOSTS[2] * Nxx_plus_2NGHOSTS[1] * Nxx_plus_2NGHOSTS[0]);
    REAL *evol_gfs  = (REAL *)malloc(sizeof(REAL) * NUM_EVOL_GFS * Nxx_plus_2NGHOSTS[2] * Nxx_plus_2NGHOSTS[1] * Nxx_plus_2NGHOSTS[0]);
    
    for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
        //auxevol_gfs[IDX4(GAMMADETGF,i0,i1,i2)] = 1.0; // Flat Space
        auxevol_gfs[IDX4(GAMMADETGF,i0,i1,i2)] = 1.0 - 1.0/(2.0+xx[0][i0]*xx[0][i0]+xx[1][i1]*xx[1][i1]+xx[2][i2]*xx[2][i2]);
    }
    
    // We now want to set up the vector potential. First, we must set the coefficients. 
    // We will use random integers between -10 and 10. For the first test, we let the 
    // Cubic coefficients remain zero. Those are a,b,c,g,h,l,p,q, and r.
    d = (double)(rand()%20-10);
    e = (double)(rand()%20-10);
    f = (double)(rand()%20-10);
    m = (double)(rand()%20-10);
    n = (double)(rand()%20-10);
    o = (double)(rand()%20-10);
    s = (double)(rand()%20-10);
    t = (double)(rand()%20-10);
    u = (double)(rand()%20-10);

    if(do_quadratic_test) {
        calculate_AD(Nxx_plus_2NGHOSTS,xx,evol_gfs);

        // We'll also calculate the exact solution for B^i
        calculate_exact_BU(Nxx_plus_2NGHOSTS,xx,auxevol_gfs);

        // And now for the numerical derivatives:
        driver_A_to_B(&params,evol_gfs,auxevol_gfs);

        printf("This test uses quadratic vector potentials, so the magnetic fields should agree to roundoff error.\\n");
        printf("Below, each row represents one point. Each column represents a component of the magnetic field.\\n");
        printf("Shown is the number of Significant Digits of Agreement, at least 13 is good, higher is better:\\n\\n");
        //Two variables for inside the loop:
        int ghost_zone_overlap;int indices[3];
        for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
            // Are we on an edge/vertex? This algorithm can probably be improved.
            ghost_zone_overlap = 0;
            indices[0] = i0;
            indices[1] = i1;
            indices[2] = i2;
            for(int dim=0;dim<3;dim++) {
                if(indices[dim]%(Nxx[dim]+NGHOSTS)<NGHOSTS) {
                    ghost_zone_overlap++;
                }
            }
            if (ghost_zone_overlap < 2) {
                // Don't print if we're on an edge or vertex
                printf("SDA: %.3f, %.3f, %.3f\\n",
                       1.0-log10(2.0*fabs(auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU0GF,i0,i1,i2)])/(fabs(auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)])+fabs(auxevol_gfs[IDX4(BU0GF,i0,i1,i2)])+1.e-15)),
                       1.0-log10(2.0*fabs(auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU1GF,i0,i1,i2)])/(fabs(auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)])+fabs(auxevol_gfs[IDX4(BU1GF,i0,i1,i2)])+1.e-15)),
                       1.0-log10(2.0*fabs(auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU2GF,i0,i1,i2)])/(fabs(auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)])+fabs(auxevol_gfs[IDX4(BU2GF,i0,i1,i2)])+1.e-15))
                       );
            }
        }
    }
    
    // Now, we'll set the cubic coefficients:
    a = (double)(rand()%20-10);
    b = (double)(rand()%20-10);
    c = (double)(rand()%20-10);
    g = (double)(rand()%20-10);
    h = (double)(rand()%20-10);
    l = (double)(rand()%20-10);
    p = (double)(rand()%20-10);
    q = (double)(rand()%20-10);
    r = (double)(rand()%20-10);
    
    // And recalculate on our initial grid:
    calculate_AD(Nxx_plus_2NGHOSTS,xx,evol_gfs);

    // We'll also calculate the exact solution for B^i
    calculate_exact_BU(Nxx_plus_2NGHOSTS,xx,auxevol_gfs);
    
    // And now for the numerical derivatives:
    driver_A_to_B(&params,evol_gfs,auxevol_gfs);
    
    // Some variables needed for the loop:
    int ghost_zone_overlap; int indices[3];
    
    char filename[100];
    sprintf(filename,"out%d-numer.txt",Nxx[0]);
    FILE *out2D = fopen(filename, "w");
    if(do_quadratic_test) {
        for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
            ghost_zone_overlap = 0;
            indices[0] = i0;
            indices[1] = i1;
            indices[2] = i2;
            for(int dim=0;dim<3;dim++) {
                if(indices[dim]%(Nxx[dim]+NGHOSTS)<NGHOSTS) {
                    ghost_zone_overlap++;
                }
            }
            if (ghost_zone_overlap < 2) {
                // We print the difference between approximate and exact numbers.
                fprintf(out2D,"%.16e\t%.16e\t%.16e\\n",
                        auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU0GF,i0,i1,i2)],
                        auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU1GF,i0,i1,i2)],
                        auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU2GF,i0,i1,i2)]
                        );
            }
        }
    }
    else {
        for(int i2=0;i2<Nxx_plus_2NGHOSTS[2];i2++) for(int i1=0;i1<Nxx_plus_2NGHOSTS[1];i1++) for(int i0=0;i0<Nxx_plus_2NGHOSTS[0];i0++) {
            ghost_zone_overlap = 0;
            indices[0] = i0;
            indices[1] = i1;
            indices[2] = i2;
            for(int dim=0;dim<3;dim++) {
                if(indices[dim]%(Nxx[dim]+NGHOSTS)<NGHOSTS) {
                    ghost_zone_overlap++;
                }
            }
            // Don't print on the edges or corners
            if (ghost_zone_overlap < 2) {
                // Only print points shared between the grids
                if (i0%2==0 && i1%2==0 && i2%2==0) {
                    // We print the difference between approximate and exact numbers.
                    fprintf(out2D,"%.16e\t%.16e\t%.16e\\n",
                            auxevol_gfs[IDX4(B_ANALYTICU0GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU0GF,i0,i1,i2)],
                            auxevol_gfs[IDX4(B_ANALYTICU1GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU1GF,i0,i1,i2)],
                            auxevol_gfs[IDX4(B_ANALYTICU2GF,i0,i1,i2)]-auxevol_gfs[IDX4(BU2GF,i0,i1,i2)]
                            );
                }
            }
        }

    }
    fclose(out2D);
}


Appending to Validation//A2B_unit_test.C


Now that we have our file, we can compile it and run the executable.

In [10]:
import time

print("Now compiling, should take ~2 seconds...\n")
start = time.time()
cmd.C_compile(os.path.join(out_dir,"A2B_unit_test.C"), os.path.join(out_dir,"A2B_unit_test"))
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")

# os.chdir(out_dir)
print("Now running...\n")
start = time.time()
# cmd.Execute(os.path.join("Stilde_flux_unit_test"))
!./Validation/A2B_unit_test 1 1 1 1
# To do a convergence test, we'll also need a second grid with twice the resolution.
!./Validation/A2B_unit_test 3 3 3 0
end = time.time()
print("Finished in "+str(end-start)+" seconds.\n\n")
# os.chdir(os.path.join("../"))


Now compiling, should take ~2 seconds...

Compiling executable...
Executing `gcc -Ofast -fopenmp -march=native -funroll-loops Validation/A2B_unit_test.C -o Validation/A2B_unit_test -lm`...
In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:1:67: error: expected ‘,’ or ‘...’ before ‘params’
 inline void compute_A2B_in_ghostzones(const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs,i0min,i0max, i1min,i1max, i2min,i2max) {
                                                                   ^~~~~~
In file included from Validation/driver_AtoB.h:2:0,
                 from Validation/A2B_unit_test.C:40:
Validation/set_Cparameters.h: In function ‘void compute_A2B_in_ghostzones(const paramstruct*)’:
Validation/set_Cparameters.h:1:18: error: ‘params’ was not declared in this scope
 const int Nxx0 = params->Nxx0;
                  ^~~~~~
In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:3:16: error: ‘i2min’ was n

In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:1:67: error: expected ‘,’ or ‘...’ before ‘params’
 inline void compute_A2B_in_ghostzones(const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs,i0min,i0max, i1min,i1max, i2min,i2max) {
                                                                   ^~~~~~
In file included from Validation/driver_AtoB.h:2:0,
                 from Validation/A2B_unit_test.C:40:
Validation/set_Cparameters.h: In function ‘void compute_A2B_in_ghostzones(const paramstruct*)’:
Validation/set_Cparameters.h:1:18: error: ‘params’ was not declared in this scope
 const int Nxx0 = params->Nxx0;
                  ^~~~~~
In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:3:16: error: ‘i2min’ was not declared in this scope
     for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
                ^~~~~
Validation/driver_AtoB.h:3:16: note: s

In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:1:67: error: expected ‘,’ or ‘...’ before ‘params’
 inline void compute_A2B_in_ghostzones(const paramstruct *restrict params,REAL *restrict in_gfs,REAL *restrict out_gfs,i0min,i0max, i1min,i1max, i2min,i2max) {
                                                                   ^~~~~~
In file included from Validation/driver_AtoB.h:2:0,
                 from Validation/A2B_unit_test.C:40:
Validation/set_Cparameters.h: In function ‘void compute_A2B_in_ghostzones(const paramstruct*)’:
Validation/set_Cparameters.h:1:18: error: ‘params’ was not declared in this scope
 const int Nxx0 = params->Nxx0;
                  ^~~~~~
In file included from Validation/A2B_unit_test.C:40:0:
Validation/driver_AtoB.h:3:16: error: ‘i2min’ was not declared in this scope
     for(int i2=i2min;i2<i2max;i2++) for(int i1=i1min;i1<i1max;i1++) for(int i0=i0min;i0<i0max;i0++) {
                ^~~~~
Validation/driver_AtoB.h:3:16: note: s

SystemExit: 1

/home/penelson/jup38/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Now that we have shown that when we use a quadratic vector potential, we get roundoff-level agreement (which is to be expected, since the finite-differencing used approximates the underlying function with a quadratic), we will use do a convergence test to show that when we can't exactly model the function, the truncation error dominates and converges to zero at the expected rate. For this, we use cubic functions for the vector potential. In the code above, we output the difference beteween the numeric and exact magnetic fields at the overlapping, non-edge, non-vertex points of two separate grids. Here, we import that data and calculate the convergence in the usual way, 
$$
k = \log_2 \left( \frac{F - F_1}{F - F_2} \right),
$$
where $k$ is the convergence order, $F$ is the exact solution, $F_1$ is the approximate solution on the coarser grid with resolution $\Delta x$, and $F_2$ is the approximate solution on the finer grid with resolution $\Delta x/2$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Data1 = np.loadtxt("out1-numer.txt")
Data2 = np.loadtxt("out3-numer.txt")

convergence = np.log(np.divide(np.abs(Data1),np.abs(Data2)))/np.log(2)
print("Convergence test: All should be approximately 2\n")
print(convergence)